In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from transformers import BertModel, BertTokenizer
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.utils.data import random_split
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # Required for tf.hub.load to work.
import pandas as pd


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:

preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")

# Load the BERT model from the Hub
encoder = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-12-h-768-a-12/versions/2", trainable=True)

In [3]:


class NewsScorePredictor(tf.keras.Model):
    def __init__(self, encoder):
        super(NewsScorePredictor, self).__init__()
        self.encoder = encoder
        self.linear = tf.keras.layers.Dense(2)  # BERT-base has 768 hidden units

    def call(self, inputs):
        outputs = self.encoder(inputs)
        pooled_output = outputs["pooled_output"]
        scores = self.linear(pooled_output)
        return scores

# Instantiate model
model = NewsScorePredictor(encoder)

raw_news_content = pd.read_csv('/kaggle/input/raw-content/raw_news_content.csv')

labels = [(russia, ukraine) for russia, ukraine in zip(raw_news_content.Russia_score, raw_news_content.Ukraine_score)]

labels = tf.constant(labels, dtype=tf.float32)

# Use the preprocessor to preprocess the text inputs
news_encoded = preprocessor(raw_news_content.raw_content.values.tolist())

# Create TensorFlow datasets for better performance
train_dataset = tf.data.Dataset.from_tensor_slices((news_encoded, labels))
train_dataset = train_dataset.batch(32).shuffle(1024)

# Optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)

# Training
history = model.fit(train_dataset, epochs=10)

# Save the model for future use
model_path = "./my_trained_model"
model.save(model_path)


Epoch 1/10
15/15 [==============================] - 386s 23s/step - loss: 15.8354
Epoch 2/10
15/15 [==============================] - 339s 23s/step - loss: 10.5530
Epoch 3/10
15/15 [==============================] - 336s 22s/step - loss: 6.3171
Epoch 4/10
15/15 [==============================] - 335s 22s/step - loss: 3.4073
Epoch 5/10
15/15 [==============================] - 332s 22s/step - loss: 2.0877
Epoch 6/10
15/15 [==============================] - 329s 22s/step - loss: 1.3923
Epoch 7/10
15/15 [==============================] - 326s 22s/step - loss: 1.0908
Epoch 8/10
15/15 [==============================] - 327s 22s/step - loss: 0.8478
Epoch 9/10
15/15 [==============================] - 324s 22s/step - loss: 0.7342
Epoch 10/10
15/15 [==============================] - 327s 22s/step - loss: 0.5316


In [4]:
def predict_text(text):
    preprocessed_text = preprocessor(tf.constant([text]))
    print(preprocessed_text)
    scores = model(preprocessed_text)
    return scores

# Example usage:
text = """To some Ukrainian forces, soldiers from the Wagner Group were the best-equipped fighters they had seen since Russia invaded last year. To others, it was their training that distinguished them: Ukrainian soldiers recalled battlefield stories of aggressive tactics or a sniper downing a drone with a single shot.
But after the short-lived mutiny led by the head of the group, Yevgeny V. Prigozhin, it is not clear whether Wagner will still be a fighting force on the battlefield with its fate now in question.
For now, the uncertain status of Wagner is bound to be a relief for Ukrainian soldiers. Though the front lines in Ukraine are likely to remain unchanged in the short term, depending on how events unfold in Russia, the Ukrainian military may be able to capitalize on the chaos and weakening morale to try to make some gains, according to independent analysts and American officials.
Still, it is too soon to determine the long-term implications of the feud between Mr. Prigozhin and the Russian military establishment, American officials said. In Bakhmut, Wagner played an outsize role in the campaign to take the eastern city, Moscow’s one major battlefield victory this year, and solidified an uneasy alliance with the Russian military — only to see the partnership break once the city was captured.
"""
print(predict_text(text))

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int32)>, 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[  101,  2000,  2070,  5969,  2749,  1010,  3548,  2013,  1996,
        10304,  2177,  2020,  1996,  2190,  1011,  6055,  7299,  2027,
         2018,  2464,  2144,  3607, 10836,  2197,  2095,  1012,  2000,
         2500,  1010,  2009,  2001,  2037,  2731,  2008,  5182,  2068,
         1024,  5969,  3548,  7383, 11686,  3441,  1997,  9376,  9887,
         2030,  1037, 17515, 22501,  1037, 18465,  2007,  103